#Web Scraping with Beautiful Soup

This tutorial will provide a very basic introduction to using the Beautiful Soup package to scrape text data from the web. 

##Installation

In a terminal, install Beautiful Soup if necessary by running <pre><code>conda install beautiful-soup</code></pre>

##Retrieving HTML

The general idea behind web scraping is to retrieve data that exists on a website, and convert it into a format that is usable for analysis. Webpages are rendered by the brower from HTML and CSS code, but much of the information included in the HTML underlying any website is not interesting to us.

We begin by reading in the source code for a given web page and creating a Beautiful Soup object with the BeautifulSoup function.

###urllib2
urllib2 is a module for working with urls, we will use it to open connections to urls and retrieve the webpage source.

In [62]:
from bs4 import BeautifulSoup
import urllib2
page = urllib2.urlopen('http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false').read()
soup = BeautifulSoup(page)
print type(soup)

<class 'bs4.BeautifulSoup'>


The soup object contains all of the HTML in the original document.

In [7]:
print soup.prettify()[6000:7000]

!--espn.us.com.nba sitezone-->
   <!--http://ad.Doubleclick.net hostUrl-->
   <script>
    if("undefined"!=typeof ad_kvps&&"undefined"!=typeof ad_u&&"undefined"!=typeof ad_counter&&"undefined"!=typeof ad_ord){if("undefined"==typeof ad_site||"undefined"==typeof ad_zone)var ad_site="espn.us.com.nba",ad_zone="nbastatisticsplayer";var ad_finalString="http://ad.Doubleclick.net/adj/"+ad_site+"/"+ad_zone+";"+ad_kvps+"pos=Background_Skin;"+ad_u+"sz=1280x946,200x800,1x1;tile="+ad_counter+";ord="+ad_ord;document.write('<div data-cb-ad-id="Background_Skin" data-cb-dfp-id="'+ad_kvps+'sz=1280x946,200x800,1x1"><script src='+ad_finalString+'>\x3c/script></div>');ad_counter++};
   </script>
   <noscript>
    <a href="http://ad.Doubleclick.net/jump/espn.us.com.nba/nbastatisticsplayer;pgtyp=nbastatisticsplayer;sp=nba;nav=true;sz=1280x946,200x800,1x1;" target="_blank">
     <img alt="" border="0" src="http://ad.Doubleclick.net/ad/espn.us.com.nba/nbastatisticsplayer;pgtyp=nbastatisticsplayer;sp=nba;nav=tr

##Parsing HTML

By "parsing HTML", we mean pulling out only the relevant tags/attributes for our analysis.  What Beautiful Soup does is provide a handy bunch of methods for doing this efficiently.

###find method

The find method will search for and return the first tag matching your corresponding search criteria, if it exists.  You can specify tag and attribute info etc.  There is also a findAll method that will return a collection of tags matching your query.

In [3]:
table_div = soup.find(id='my-players-table')
print table_div.prettify()[0:4000]

<div class="col-main" id="my-players-table">
 <div class="mod-container mod-table">
  <div class="mod-header stathead">
   <h4>
    Points Per Game Leaders - All Players
   </h4>
  </div>
  <div class="mod-content">
   <table cellpadding="3" cellspacing="1" class="tablehead">
    <tr align="right" class="colhead">
     <td align="left" style="width:20px;">
      RK
     </td>
     <td align="left">
      PLAYER
     </td>
     <td align="left">
      TEAM
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">
       GP
      </a>
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">
       MPG
      </a>
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">
       PTS

Notice that the object returned by the find method is just another inner HTML structure, which we can step through just like we would have with the original soup object.  We've gone to the location in the webpage where the table that we seek starts, now we can use find again to get to the table data.

In [9]:
table = table_div.find("table")
print table.prettify()

<table cellpadding="3" cellspacing="1" class="tablehead">
 <tr align="right" class="colhead">
  <td align="left" style="width:20px;">
   RK
  </td>
  <td align="left">
   PLAYER
  </td>
  <td align="left">
   TEAM
  </td>
  <td>
   <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">
    GP
   </a>
  </td>
  <td>
   <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">
    MPG
   </a>
  </td>
  <td>
   <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">
    PTS
   </a>
  </td>
  <td>
   <span title="Field Goals Made-Attempted Per Game">
    FGM-FGA
   </span>
  </td>
  <td>
   <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">
    FG%
   </a>
  </td>
  <td

###Searching by Attributes

Now that we have the table object, we need to step through the rows.  First we'll find the header row so we can populate what the field names will be in our data.  Here we're searching for tags under the table tag whose class attritbute is "colhead".  

In [10]:
table_head = table.find(attrs={"class":'colhead'})
print table_head.prettify()

<tr align="right" class="colhead">
 <td align="left" style="width:20px;">
  RK
 </td>
 <td align="left">
  PLAYER
 </td>
 <td align="left">
  TEAM
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">
   GP
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">
   MPG
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">
   PTS
  </a>
 </td>
 <td>
  <span title="Field Goals Made-Attempted Per Game">
   FGM-FGA
  </span>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">
   FG%
  </a>
 </td>
 <td>
  <span title="3-Point Field Goals Made-Attempted Per Game">
   3PM-3PA
  </span>
 </td>
 <t

Now we find the actual values by searching for the 'td' tags, which is the tag for table data.

In [11]:
header_cols = table_head.findAll('td')
print header_cols

[<td align="left" style="width:20px;">RK</td>, <td align="left">PLAYER</td>, <td align="left">TEAM</td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td>, <td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td>, <td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/threePointFieldGoalPct/qualified/false" t

Finally, we step through these columns and save them to a list to be used later.  We'll ignore the rank column (RK) because that doesn't give us anything we want later.  We also separate the **PLAYER** column into **PLAYER** and **POSITION**.

In [12]:
cols = []
for header_col in header_cols:
    val = header_col.string
    if val != 'RK':
        cols.append(val)
    if val == 'PLAYER':
        cols.append('POSITION')
print cols

[u'PLAYER', 'POSITION', u'TEAM', u'GP', u'MPG', u'PTS', u'FGM-FGA', u'FG%', u'3PM-3PA', u'3P%', u'FTM-FTA', u'FT%']


###Stepping Through a Table

The table rows are indicated by the tag 'tr'.  Again we can find them all and iterate through them.  Within each row we iterate through the respective columns.

In [13]:
table_rows = table.findAll('tr')
print table_rows

[<tr align="right" class="colhead"><td align="left" style="width:20px;">RK</td><td align="left">PLAYER</td><td align="left">TEAM</td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td><td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td><td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/threePointFieldGoalPct/qu

We will save our results in 2 different ways to demonstrate how we can handle both.  The first will be a list of dicts where the key is the field name and the value is the field value.  The second will just be a list of lists of stats with no field name values (we've already defined them earlier).

In [14]:
players_stats_dicts = []
players_stats_array = []
for row in table_rows:
    if row.attrs['class'][0]=='colhead':
        continue
    player_stats = []
    row_cols = row.find_all('td')
    col_vals = []
    player_col = row_cols[1]
    player_name = player_col.find('a').string
    player_position = player_col.contents[1]
    # Split by ' ' makes an array
    player_position = player_position.split(' ')[1]
    player_stats.append(player_name)
    player_stats.append(player_position)
    for i in range(2, len(row_cols)):
        stat = row_cols[i].string
        player_stats.append(stat)
    players_stats_array.append(player_stats)
    player_stats = dict(zip(cols, player_stats))
    # zip takes 2 lists, outputs pairs
    players_stats_dicts.append(player_stats)
print players_stats_dicts[0:5]        

[{u'FGM-FGA': u'9.4-22.0', u'MPG': u'34.4', u'FTM-FTA': u'8.1-9.8', u'FG%': u'.426', u'GP': u'67', u'PLAYER': u'Russell Westbrook', u'FT%': u'.835', u'TEAM': u'OKC', u'3PM-3PA': u'1.3-4.3', 'POSITION': u'PG', u'PTS': u'28.1', u'3P%': u'.299'}, {u'FGM-FGA': u'8.0-18.1', u'MPG': u'36.8', u'FTM-FTA': u'8.8-10.2', u'FG%': u'.440', u'GP': u'81', u'PLAYER': u'James Harden', u'FT%': u'.868', u'TEAM': u'HOU', u'3PM-3PA': u'2.6-6.9', 'POSITION': u'SG', u'PTS': u'27.4', u'3P%': u'.375'}, {u'FGM-FGA': u'8.8-17.3', u'MPG': u'33.8', u'FTM-FTA': u'5.4-6.3', u'FG%': u'.510', u'GP': u'27', u'PLAYER': u'Kevin Durant', u'FT%': u'.854', u'TEAM': u'OKC', u'3PM-3PA': u'2.4-5.9', 'POSITION': u'SF', u'PTS': u'25.4', u'3P%': u'.403'}, {u'FGM-FGA': u'9.0-18.5', u'MPG': u'36.1', u'FTM-FTA': u'5.4-7.7', u'FG%': u'.488', u'GP': u'69', u'PLAYER': u'LeBron James', u'FT%': u'.710', u'TEAM': u'CLE', u'3PM-3PA': u'1.7-4.9', 'POSITION': u'SF', u'PTS': u'25.3', u'3P%': u'.354'}, {u'FGM-FGA': u'9.4-17.6', u'MPG': u'36.1'

Here we've used the zip function to combine pairs of lists into tuples, and then transformed that into a dict to get a dictionary of FIELD --> VALUE for every player in the table.

Beautiful Soup has many other features, including the ability to step up, down, and sideways in the HTML tree and basically search for any tags, attributes, or values.  For more, take a look at the [Documentation](http://www.crummy.com/software/BeautifulSoup/bs4/doc/)

##Load into Pandas

Let's load our scraped data into Pandas and take a look at it.  Here is the first way we can do it, simply directly from the dictionary we defined.

In [15]:
import pandas as pd
import numpy as np

df = pd.DataFrame.from_dict(players_stats_dicts)
df.head()

,3P%,3PM-3PA,FG%,FGM-FGA,FT%,FTM-FTA,GP,MPG,PLAYER,POSITION,PTS,TEAM
0,.299,1.3-4.3,.426,9.4-22.0,.835,8.1-9.8,67,34.4,Russell Westbrook,PG,28.1,OKC
1,.375,2.6-6.9,.440,8.0-18.1,.868,8.8-10.2,81,36.8,James Harden,SG,27.4,HOU
2,.403,2.4-5.9,.510,8.8-17.3,.854,5.4-6.3,27,33.8,Kevin Durant,SF,25.4,OKC
3,.354,1.7-4.9,.488,9.0-18.5,.710,5.4-7.7,69,36.1,LeBron James,SF,25.3,CLE
4,.083,0.0-0.2,.535,9.4-17.6,.805,5.5-6.8,68,36.1,Anthony Davis,PF,24.4,NO


Now here is a 2nd way we can do it.  We convert the 2D stats array into a numpy array and create a Pandas dataframe from it along with the list of column headers we defined earlier.

In [16]:
np_array = np.array(players_stats_array)
df = pd.DataFrame(np_array, columns=cols)
df.head()

,PLAYER,POSITION,TEAM,GP,MPG,PTS,FGM-FGA,FG%,3PM-3PA,3P%,FTM-FTA,FT%
0,Russell Westbrook,PG,OKC,67,34.4,28.1,9.4-22.0,.426,1.3-4.3,.299,8.1-9.8,.835
1,James Harden,SG,HOU,81,36.8,27.4,8.0-18.1,.440,2.6-6.9,.375,8.8-10.2,.868
2,Kevin Durant,SF,OKC,27,33.8,25.4,8.8-17.3,.510,2.4-5.9,.403,5.4-6.3,.854
3,LeBron James,SF,CLE,69,36.1,25.3,9.0-18.5,.488,1.7-4.9,.354,5.4-7.7,.710
4,Anthony Davis,PF,NO,68,36.1,24.4,9.4-17.6,.535,0.0-0.2,.083,5.5-6.8,.805


##Exercise

The goal of this exercise is to combine the scoring and assists statistics for every player in the NBA in 2014-2015.  The end result will have them in a pandas dataframe with the fields from both pages for every player.

The general steps should be as follows:
- Create a function get_cols that retrieves the names of the header columns given a table element (skip the ranks, split the positions)
- Create a function get_data that retrieves the actual table data given a table element (skip the ranks, split the positions).  You can use either the dict approach or the numpy array approach.
- Write a python loop to loop through the various pages and call these functions on the appropriate urls so that you can retrieve every player (rather than just the top few).
- Repeat the above on both the scoring and assists URLs to get a pandas dataframe for both of them
- Use the pandas.DataFrame.join() function to join your 2 pandas dataframes together and get a total result

In [63]:
# There are two approaches in the player_stats functions
def get_cols(table):
    table_head = table.find(attrs={"class":'colhead'})
    header_cols = table_head.findAll('td')
    cols = []
    for header_col in header_cols:
        val = header_col.string
        if val != 'RK':
            cols.append(val)
        if val == 'PLAYER':
            cols.append('POSITION')
    return cols
    
def get_data(table, cols):
    table_rows = table.findAll('tr')
    players_stats_dicts = []
    for row in table_rows:
        if row.attrs['class'][0]=='colhead':
            continue
        player_stats = []
        row_cols = row.find_all('td')
        col_vals = []
        player_col = row_cols[1]
        player_name = player_col.find('a').string
        player_position = player_col.contents[1]
        # Split by ' ' makes an array
        player_position = player_position.split(' ')[1]
        player_stats.append(player_name)
        player_stats.append(player_position)
        for i in range(2, len(row_cols)):
            stat = row_cols[i].string
            player_stats.append(stat)
        player_stats = dict(zip(cols, player_stats))
        # zip takes 2 lists, outputs pairs
        players_stats_dicts.append(player_stats)
    return players_stats_dicts


In [65]:
import pandas as pd
import numpy as np

url = 'http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/'
all_stats_dicts = []

for i in range(0, 13):
    page_number = str(i * 40 + 1)
    page = urllib2.urlopen(url + page_number).read()
    soup = BeautifulSoup(page)
    table = soup.find(id='my-players-table').find("table")
    cols = get_cols(table)
    players_stats_dicts = get_data(table, cols)
    all_stats_dicts.append(players_stats_dicts)
    flat_stats_dicts = [val for sublist in all_stats_dicts for val in sublist]

df = pd.DataFrame.from_dict(flat_stats_dicts)
df.head(42)

,3P%,3PM-3PA,FG%,FGM-FGA,FT%,FTM-FTA,GP,MPG,PLAYER,POSITION,PTS,TEAM
0,.299,1.3-4.3,.426,9.4-22.0,.835,8.1-9.8,67,34.4,Russell Westbrook,PG,28.1,OKC
1,.375,2.6-6.9,.440,8.0-18.1,.868,8.8-10.2,81,36.8,James Harden,SG,27.4,HOU
2,.403,2.4-5.9,.510,8.8-17.3,.854,5.4-6.3,27,33.8,Kevin Durant,SF,25.4,OKC
3,.354,1.7-4.9,.488,9.0-18.5,.710,5.4-7.7,69,36.1,LeBron James,SF,25.3,CLE
4,.083,0.0-0.2,.535,9.4-17.6,.805,5.5-6.8,68,36.1,Anthony Davis,PF,24.4,NO
5,.341,1.5-4.5,.444,9.0-20.2,.797,4.7-5.9,40,35.7,Carmelo Anthony,SF,24.2,NY
6,.250,0.0-0.1,.467,8.4-18.1,.782,7.2-9.2,59,34.1,DeMarcus Cousins,C,24.1,SAC
7,.443,3.6-8.1,.487,8.2-16.8,.914,3.9-4.2,80,32.7,Stephen Curry,PG,23.8,GS
8,.352,0.5-1.5,.466,9.3-19.9,.845,4.3-5.1,71,35.4,LaMarcus Aldridge,PF,23.4,POR
9,.293,1.5-5.3,.373,7.6-20.4,.813,5.6-6.9,35,34.5,Kobe Bryant,SG,22.3,LAL


In [85]:
url2 = 'http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgAssists/qualified/false/count/'
all_assists_dicts = []

for i in range(0, 13):
    page_number = str(i * 40 + 1)
    page = urllib2.urlopen(url2 + page_number).read()
    soup = BeautifulSoup(page)
    table = soup.find(id='my-players-table').find("table")
    cols = get_cols(table)
    players_stats_dicts = get_data(table, cols)
    all_assists_dicts.append(players_stats_dicts)
    flat_stats_dicts = [val for sublist in all_stats_dicts for val in sublist]

df2 = pd.DataFrame.from_dict(flat_stats_dicts)
df2.head(42)

,3P%,3PM-3PA,FG%,FGM-FGA,FT%,FTM-FTA,GP,MPG,PLAYER,POSITION,PTS,TEAM
0,.299,1.3-4.3,.426,9.4-22.0,.835,8.1-9.8,67,34.4,Russell Westbrook,PG,28.1,OKC
1,.375,2.6-6.9,.440,8.0-18.1,.868,8.8-10.2,81,36.8,James Harden,SG,27.4,HOU
2,.403,2.4-5.9,.510,8.8-17.3,.854,5.4-6.3,27,33.8,Kevin Durant,SF,25.4,OKC
3,.354,1.7-4.9,.488,9.0-18.5,.710,5.4-7.7,69,36.1,LeBron James,SF,25.3,CLE
4,.083,0.0-0.2,.535,9.4-17.6,.805,5.5-6.8,68,36.1,Anthony Davis,PF,24.4,NO
5,.341,1.5-4.5,.444,9.0-20.2,.797,4.7-5.9,40,35.7,Carmelo Anthony,SF,24.2,NY
6,.250,0.0-0.1,.467,8.4-18.1,.782,7.2-9.2,59,34.1,DeMarcus Cousins,C,24.1,SAC
7,.443,3.6-8.1,.487,8.2-16.8,.914,3.9-4.2,80,32.7,Stephen Curry,PG,23.8,GS
8,.352,0.5-1.5,.466,9.3-19.9,.845,4.3-5.1,71,35.4,LaMarcus Aldridge,PF,23.4,POR
9,.293,1.5-5.3,.373,7.6-20.4,.813,5.6-6.9,35,34.5,Kobe Bryant,SG,22.3,LAL


In [87]:
dfAll = df.join(df2, 'PLAYER', how='left', lsuffix='', rsuffix='assist', sort=True)
dfAll.head(102)

,3P%,3PM-3PA,FG%,FGM-FGA,FT%,FTM-FTA,GP,MPG,PLAYER,POSITION,...,FG%assist,FGM-FGAassist,FT%assist,FTM-FTAassist,GPassist,MPGassist,PLAYERassist,POSITIONassist,PTSassist,TEAMassist
320,.263,0.6-2.2,.372,2.0-5.3,.667,0.6-0.9,26,12.5,A.J. Price,PG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,.387,1.5-3.8,.421,4.2-10.0,.833,1.8-2.1,82,23.0,Aaron Brooks,PG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
316,.271,0.3-1.0,.447,2.0-4.4,.721,0.9-1.3,47,17.0,Aaron Gordon,PF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
253,.111,0.0-0.3,.414,2.8-6.9,.652,0.9-1.4,32,23.1,Adreian Payne,PF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,.306,0.1-0.5,.538,6.8-12.7,.759,1.4-1.9,76,30.5,Al Horford,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,.400,0.0-0.1,.481,7.5-15.5,.655,1.7-2.5,65,30.6,Al Jefferson,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,.274,0.5-1.7,.412,2.0-4.8,.712,1.1-1.6,74,18.5,Al-Farouq Aminu,SF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
229,.348,1.0-2.8,.443,2.6-5.9,.812,1.1-1.4,74,23.6,Alan Anderson,SG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,.382,1.0-2.5,.403,4.5-11.1,.822,3.9-4.8,27,33.3,Alec Burks,PG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
478,.000,0.0-0.0,.250,0.2-0.8,1.000,0.4-0.4,5,2.8,Alex Kirk,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
